In [1]:
from dataclasses import dataclass

from lark import Lark


In [16]:
grammar = "".join(open("Voja4 Grammar.lark", "r").readlines())
assembly_code = "".join(open("Duty Cycle.voj4", mode="r").readlines())

parser = Lark(grammar, propagate_positions=True)

@dataclass
class HexLine:
    line_num: int
    label: str
    hex: str
    assembly: str
    comment: str
    def __repr__(self):
        return f"{self.line_num:04d} {self.label:<8s} {self.hex[0:4]} {self.hex[4:8]} {self.hex[8:12]}   {self.assembly}   {self.comment}".strip()

class HexCode:
    hexlines: list[HexLine] = []
    def lines(self):
        return len(self.hexlines)
    def add(self, label, hex, assembly, comment):
        self.hexlines.append(HexLine(self.lines(), label, hex, assembly, comment))

tree = parser.parse(assembly_code)

print(tree.pretty())

hexcode = HexCode()
labels = {}
label = ""
for line in tree.children:
    assert line.data == "line"
    num_parts = len(line.children)
    match num_parts:
        case 1:
            part = line.children[0]
            if part.data == "comment":
                continue
            else:
                body = part
                comment_text = ""
        case 2:
            body, comment = line.children
            assert comment.data == "comment"
            comment_text = comment.children[0]
        case _:
            raise "Bad parsing"
    body_type = body.data
    match body_type:
        case "label":
            label = body.children[0].value
            labels[label] = hexcode.lines()
        case "goto" | "gosub":
            target = body.children[0].value
            hexcode.add(label=label, hex=f"{body_type} hi", assembly=target, comment="")
            hexcode.add(label="", hex=f"{body_type} lo", assembly=target, comment="")
            label = ""
        case x if x.startswith("inst"):
            opcode = x[5:].replace("_", "")
            for param in body.children:
                param_type, *param_code = param.data.split("_")
                match param_type:
                    case "reg2" | "reg4" | "test" | "jump":
                        opcode += "".join(param_code)
                    case "num2":
                        opcode += bin(int(param.children[0]))[2:].zfill(2)
                    case "num4":
                        opcode += bin(int(param.children[0], 0))[2:].zfill(4)
                    case "num8":
                        value = int(param.children[0], 0)
                        value = value + 256 if value < 0 else value
                        opcode += bin(value)[2:].zfill(8)

            hexcode.add(label, opcode, assembly_code[line.meta.start_pos:line.meta.end_pos], "")
            label = ""
        case _:
            print("oops")
            print(line_body)
            print(line_type)

#2nd pass to put numeric addresses into the jumps
for line_body in hexcode.hexlines:
    match(line_body.hex):
        case "goto lo":
            target = line_body.assembly
            address = bin(labels[target] & 15)[2:].zfill(4)
            line_body.hex = f"10011101{address}"
            line_body.assembly = f"MOV PCL, {target}"
        case "gosub lo":
            target = line_body.assembly
            address = bin(labels[target] & 15)[2:].zfill(4)
            line_body.hex = f"10011100{address}"
            line_body.assembly = f"MOV JSR, {target}"
        case "goto hi" | "gosub hi":
            target = line_body.assembly
            address = bin(labels[target] >> 4)[2:].zfill(8)
            line_body.hex = f"1110{address}"
            line_body.assembly = f"MOV PC, {target}"

#print out the result
for line in hexcode.hexlines:
    print(line)

start
  line
    comment	 Register use:
  line
    comment	 R0 - value to draw
  line
    comment	 R1 - page to use
  line
    comment	 R2 - row to use
  line
    comment	 R3 - lower nibble of wait time
  line
    comment	 R4 - upper nibble of wait time
  line
    comment	 R6 - loop lower nibble
  line
    comment	 R7 - loop upper nibble
  line
    label	config
  line
    inst_1001
      reg4_0001
      num4	2
    comment	 page
  line
    inst_1001
      reg4_0010
      num4	0
    comment	 row to draw
  line
    inst_1001
      reg4_0011
      num4	15
    comment	 lower nibble of wait time
  line
    inst_1001
      reg4_0100
      num4	1
    comment	 upper nibble of wait time
  line
    inst_1001
      reg4_0101
      num4	1
    comment	 bit to toggle
  line
    label	setup
  line
    inst_1001
      reg4_0000
      num4	0
  line
    inst_1100
      num8	0xF1
    comment	 set clock speed
  line
    inst_1000
      reg4_0000
      reg4_0001
  line
    inst_1100
      num8	0xF0
    comm